# Development of a h2k parser
I adopted Adam's h2k parser to extract the geometry data from a h2k file.

In [1]:
import xmlschema
from operator import itemgetter

In [2]:
h2k_schema = xmlschema.XMLSchema('../../../schemas/h2k/H2k Schema.xsd')
assert h2k_schema.is_valid('../../data/h2k/ERS-1032.H2K'), "Not a valid h2k file"
h2k_dict = h2k_schema.to_dict('../../data/h2k/ERS-1032.H2K')

In [3]:
h2k_dict.keys()

dict_keys(['@xmlns:xsd', '@xmlns:xsi', '@{http://www.w3.org/XML/1998/namespace}lang', '@uiUnits', 'Version', 'Application', 'ProgramInformation', 'House', 'Codes', 'FuelCosts', 'AllResults', 'Program'])

## Get h2k file identifications


In [4]:
#version_major = h2k_dict['Version']['@major']
#version_minor = h2k_dict['Version']['@minor']
major, minor = itemgetter('@major','@minor')(h2k_dict['Version'])
print("Version major= {} and minor= {}".format(major, minor))

Version major= 1 and minor= 3


In [5]:
fileID = h2k_dict['ProgramInformation']['File']['Identification']
print(fileID)

ERS-1032


In [6]:
weather_prov = h2k_dict['ProgramInformation']['Weather']['Region']['English']
weather_loc = h2k_dict['ProgramInformation']['Weather']['Location']['English']
#weather_frostdepth = float(h2k_dict['ProgramInformation']['Weather']['@depthOfFrost'])
#weather_hdd = float(h2k_dict['ProgramInformation']['Weather']['@heatingDegreeDay'])
frostdepth, hdd = itemgetter('@depthOfFrost', '@heatingDegreeDay')(h2k_dict['ProgramInformation']['Weather'])
print("Region: {}, Location: {}, Depth of Frost= {}, HDD= {}".format(weather_prov, weather_loc, float(frostdepth), float(hdd)))
h2k_dict['ProgramInformation']['Weather']

Region: YUKON TERRITORY, Location: WHITEHORSE, Depth of Frost= 1.2192, HDD= 6580.0


{'@depthOfFrost': Decimal('1.2192'),
 '@heatingDegreeDay': Decimal('6580'),
 '@library': 'Wth110.dir',
 'Region': {'@code': '11',
  'English': 'YUKON TERRITORY',
  'French': 'TERRITOIRE DU YUKON'},
 'Location': {'@code': '69', 'English': 'WHITEHORSE', 'French': 'WHITEHORSE'}}

# Get h2k specifications

In [7]:
hse_type, hse_code = itemgetter('English', '@code')(h2k_dict['House']['Specifications']['HouseType'])
plan_type, plan_code = itemgetter('English', '@code')(h2k_dict['House']['Specifications']['PlanShape'])
storeys, storeys_code = itemgetter('English', '@code')(h2k_dict['House']['Specifications']['Storeys'])
facingDirection, facingDirection_code = itemgetter('English', '@code')(h2k_dict['House']['Specifications']['FacingDirection'])
thermalMass, thermalMass_code = itemgetter('English', '@code')(h2k_dict['House']['Specifications']['ThermalMass'])
print("House type: {}, and type code={}".format(hse_type, hse_code))
print("Plan shape: {}, and shape code={}".format(plan_type, plan_code))
print("Storeys: {}, and storeys code={}".format(storeys, storeys_code))
print("Facing direction: {}, and facing direction code={}".format(facingDirection, facingDirection_code))
print("Thermal mass: {}, and thermal mass code={}".format(thermalMass, thermalMass_code))

House type: Row house, middle unit, and type code=8
Plan shape: Rectangular, and shape code=1
Storeys: Two storeys, and storeys code=3
Facing direction: North, and facing direction code=5
Thermal mass: Light, wood frame, and thermal mass code=1


In [8]:
# Alternatively we can dump all the data with less code
house_specs = ['HouseType', 'PlanShape', 'Storeys', 'FacingDirection', 'ThermalMass', 'SoilCondition', 'WaterLevel']
for spec in house_specs:
    print(spec, itemgetter('English', '@code')(h2k_dict['House']['Specifications'][spec]))

HouseType ('Row house, middle unit', '8')
PlanShape ('Rectangular', '1')
Storeys ('Two storeys', '3')
FacingDirection ('North', '5')
ThermalMass ('Light, wood frame', '1')
SoilCondition ('Normal conductivity (dry sand, loam, clay)', '1')
WaterLevel ('Normal (7-10m/23-33ft)', '2')


In [9]:
vintage_bins = {2:1920,3:1925,4:1935,5:1945,6:1955,7:1965,8:1975,9:1985,10:1995,11:2005}
vintage_code = h2k_dict['House']['Specifications']['YearBuilt']['@code']
if vintage_code in vintage_bins.keys():
    vintage = vintage_bins[vintage_code]
else:
    vintage = h2k_dict['House']['Specifications']['YearBuilt']['@value']
print(vintage)

2017


In [10]:
ag_HeatedArea, bg_HeatedArea = itemgetter('@aboveGrade', '@belowGrade')(h2k_dict['House']['Specifications']['HeatedFloorArea'])
print("Above grade floor area={} & below grade floor area={}".format(ag_HeatedArea,bg_HeatedArea))

Above grade floor area=185.2 & below grade floor area=91.1


In [11]:
roofCavityACH = float(h2k_dict['House']['Specifications']['RoofCavity']['@ventilationRate'])
roofCavityVOL = float(h2k_dict['House']['Specifications']['RoofCavity']['@volume'])
# gable ends
gableEndArea = float(h2k_dict['House']['Specifications']['RoofCavity']['GableEnds']['@area'])
gableEndSheatingCode = int(h2k_dict['House']['Specifications']['RoofCavity']['GableEnds']['SheatingMaterial']['@code'])
gableEndSheatingRSI = float(h2k_dict['House']['Specifications']['RoofCavity']['GableEnds']['SheatingMaterial']['@value'])

gableEndSheatingExteriorCode = int(h2k_dict['House']['Specifications']['RoofCavity']['GableEnds']['ExteriorMaterial']['@code'])
gableEndSheatingExteriorRSI = float(h2k_dict['House']['Specifications']['RoofCavity']['GableEnds']['ExteriorMaterial']['@value'])
# Sloped roof
slopedRoofArea = float(h2k_dict['House']['Specifications']['RoofCavity']['SlopedRoof']['@area'])
slopedRoofSheatingCode = int(h2k_dict['House']['Specifications']['RoofCavity']['SlopedRoof']['SheatingMaterial']['@code'])
slopedRoofSheatingRSI = float(h2k_dict['House']['Specifications']['RoofCavity']['SlopedRoof']['SheatingMaterial']['@value'])

slopedRoofSheatingExteriorCode = int(h2k_dict['House']['Specifications']['RoofCavity']['SlopedRoof']['RoofingMaterial']['@code'])
slopedRoofSheatingExteriorRSI = float(h2k_dict['House']['Specifications']['RoofCavity']['SlopedRoof']['RoofingMaterial']['@value'])


## Get H2K Temperatures 

In [12]:
(DayHeatingSetpoint, NightHeatingSetpoint, NightSetbackDur, CoolingSetpoint) = itemgetter('@daytimeHeatingSetPoint', '@nighttimeHeatingSetPoint', '@nighttimeSetbackDuration', '@coolingSetPoint')\
                                                                            (h2k_dict['House']['Temperatures']['MainFloors'])
AllowedTempRise = {1:0, 2:2.8, 3:5.5}
AllowTempRise = AllowedTempRise[int(h2k_dict['House']['Temperatures']['MainFloors']['AllowableRise']['@code'])]
print(DayHeatingSetpoint, NightHeatingSetpoint, NightSetbackDur, CoolingSetpoint)

21 18 8 25


In [13]:
isBasementHeated, isBasementCooled, isBasementSepThermo, isBasementAUnit = itemgetter('@heated', '@cooled', '@separateThermostat', '@basementUnit')\
                                                                            (h2k_dict['House']['Temperatures']['Basement'])
basementHeatSetpoint = float(h2k_dict['House']['Temperatures']['Basement']['@heatingSetPoint'])
print(isBasementHeated, isBasementCooled, isBasementSepThermo, isBasementAUnit,basementHeatSetpoint)

True False True False 19.0


In [14]:
equipmentHeatSetpoint, equipmentCoolSetpoint = itemgetter('@heatingSetPoint', '@coolingSetPoint')(h2k_dict['House']['Temperatures']['Equipment'])

In [15]:
isCrawlSpaceHeated = bool(h2k_dict['House']['Temperatures']['Crawlspace']['@heated'])
crawlSpaceHeatSetpoint = float(h2k_dict['House']['Temperatures']['Crawlspace']['@heatingSetPoint'])